In [1]:
pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 5.2 MB/s 
     |████████████████████████████████| 78 kB 4.3 MB/s 
     |████████████████████████████████| 4.7 MB 44.5 MB/s 
     |████████████████████████████████| 237 kB 44.0 MB/s 
     |████████████████████████████████| 164 kB 53.7 MB/s 
     |████████████████████████████████| 182 kB 50.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=df86289caa6f4b7f2c21a729506e6a512bbe8652d7989ef2f7abdda18ac32dfd
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
'''def main():
	st.title("Covid19 Tweets Dashboard")
	st.subheader("Tweets Analysis")
	menu = ["Home","About"]
	choice = st.sidebar.selectbox('Menu',menu)
	if choice == 'Home':
		st.subheader("Covid19 Tweets Dashboard")	

if __name__ == '__main__':
	main()'''

'def main():\n\tst.title("Covid19 Tweets Dashboard")\n\tst.subheader("Tweets Analysis")\n\tmenu = ["Home","About"]\n\tchoice = st.sidebar.selectbox(\'Menu\',menu)\n\tif choice == \'Home\':\n\t\tst.subheader("Covid19 Tweets Dashboard")\t\n\nif __name__ == \'__main__\':\n\tmain()'

In [2]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=67fe11ff22bb9916f287aa56fdfc12bc18214721aace993848c9ad4f4e5d279f
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [4]:
!pip install plotly-express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.379s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [10]:
!pip install streamlit-autorefresh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 345 kB 4.5 MB/s 


In [11]:
%%writefile app.py
import plotly.express as px
import pickle
import streamlit as st
from streamlit_autorefresh import st_autorefresh
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, timedelta
import emoji as ej
import re
import string
# NLTK imports
import nltk
nltk.download('punkt')
# Download stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer 
st.set_option('deprecation.showPyplotGlobalUse', False) 

st.set_page_config(page_title = "Covid19 Tweets Dashboard",
                   page_icon = ":bar_chart:",
                   layout = "wide")

st.title("Covid19 Tweets Dashboard")
st.markdown("---")

st_autorefresh(interval= 1 * 60 * 1000, key="dataframerefresh")

def extract_tweets():
    n = st.number_input("Enter the number of tweets to be extracted for last 1 day:", min_value=1, step=1)
    tweets_list1 = []
    since_date = int((datetime.now()+timedelta(days=-1)).timestamp())
    until_date = int((datetime.now()).timestamp())
    i=-1
    try:
      n = abs(int(n))
    except Exception as e:
      print("Enter a valid number\n",e)
      raise
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(
            query='(covid OR covid19 OR coronavirus) min_replies:1 lang:en -filter:links -filter:replies until_time:{0} since_time:{1}'.format(until_date,since_date)
        ).get_items()
    ):
        tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.username])
        if((i + 1)==n):
            break
    else:
        print("Only {0} tweets available for past 24 hours".format(i+1))
    tweets_df1 = pd.DataFrame(
        tweets_list1, columns=["Datetime", "Tweet Id", "Text", "Username"]
    )
    return tweets_df1

df = extract_tweets()

def emoji_to_text(text):
    if type(text) != float:
        return (ej.demojize(text)).replace(":", " ").replace("_"," ")
    else:
        return text.replace(":", " ").replace("_"," ")


def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    tweet = re.sub('#','',tweet)
    #emoj = re.compile("["
        #u"\U0001F600-\U0001F64F"  # emoticons
        #u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        #u"\U0001F680-\U0001F6FF"  # transport & map symbols
        #u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        #u"\U00002500-\U00002BEF"  # chinese char
        #u"\U00002702-\U000027B0"
        #u"\U00002702-\U000027B0"
        #u"\U000024C2-\U0001F251"
        #u"\U0001f926-\U0001f937"
        #u"\U00010000-\U0010ffff"
        #u"\u2640-\u2642" 
        #u"\u2600-\u2B55"
        #u"\u200d"
        #u"\u23cf"
        #u"\u23e9"
        #u"\u231a"
        #u"\ufe0f"  # dingbats
        #u"\u3030"
        #              "]+", re.UNICODE)
    #return re.sub(emoj, '', tweet)
    return tweet

def remove_mult_spaces(text): # remove multiple spaces
  return re.sub("\s\s+" , " ", text)

contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "i'd": "i would", "i'd've": "i would have","i'll": "i will",
                     "i'll've": "i will have","i'm": "i am","i've": "i have", "isn't": "is not",
                     "it'd": "it would","it’s":"it is","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are","what’s":"what is" ,"what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

def CleanFurther(Text):
  Text = re.sub(" \d+", " ", Text) # Numbers 
  Text = re.sub(r'(?:^| )\w(?:$| )', ' ', Text).strip() #Single letters 
  Text = re.sub('[%s]' % re.escape("""!"#$%&'’()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', Text)  # remove punctuations
  Text = re.sub('\s+', ' ',Text) #Extra whitespaces
  return Text

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def data_processing(df):
  # drop duplicates
  new_df = df.drop_duplicates(subset=['Tweet Id'], keep='first')
  new_df['text'] = new_df.Text.str.lower()
  new_df = new_df.drop(columns='Text')
  new_df = new_df[(new_df.text.str.contains('covid'))|(new_df.text.str.contains('coronavirus'))] #to ensure tweets have a reference to covid and are not extrcaed due to just the username having reference to covid
  new_df = new_df.reset_index(drop=True)
  new_df['text_ett'] = new_df['text'].apply(emoji_to_text)
  new_df['pr_text'] = new_df.text_ett.apply(lambda x:remove_usernames_links(x)).values
  new_df['pr_text']=new_df['pr_text'].apply(lambda x:expand_contractions(x))
  new_df['pr_text']=new_df['pr_text'].apply(lambda x:CleanFurther(x))
  new_df['pr_text_swr'] = new_df['pr_text'].apply(lambda x: ' '.join([str(w).strip('.') for w in word_tokenize(x) if not w in stop_words if not w in string.punctuation if not w in ['``',"''"]]))
  new_df['pr_text_swr_lemma'] = new_df.pr_text_swr.apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.split()]))
  return new_df

df= data_processing(df)

#st.sidebar.header("Filter the time duration here:")
#time = st.sidebar.multiselect(
#	"Select the timeframe:",
#	options = ["10 min", "30 min", "1 hour", "2 hours", "6 hours", "12 hours", "1 Day", "1 Week", "1 month", "3 months"],
#	default = "6 hours"
#)

#df_selection = df.query()

#cv = CountVectorizer(min_df=3, max_df=0.4, binary=False, ngram_range=(1,2), max_features=4000)

cv= pickle.load(open('/content/drive/MyDrive/finalized_model_cv1_08.sav', 'rb'))

loaded_model = pickle.load(open('/content/drive/MyDrive/finalized_model_mnb_08.sav', 'rb'))
df.pr_text_swr_lemma=cv.transform(df.pr_text_swr_lemma)
#result = loaded_model.predict(df.pr_text_swr_lemma)
df['Sentiment']=loaded_model.predict(df.pr_text_swr_lemma)

df1 = df[['Datetime', 'Tweet Id',	'Username',	'text', 'pr_text_swr_lemma','Sentiment']].copy()

replacement_mapping_dict = {
    0 : "Negative",
    1 : "Neutral",
    2 : "Positive"
}

df1['Sentiment'] = df1['Sentiment'].replace(replacement_mapping_dict)

st.header(":globe_with_meridians: Extracted Tweets")
st.dataframe(df1, use_container_width=True)
st.markdown("---")

st.header(":bar_chart: Tweets Analysis")
st.markdown("##")

sentiment_count = df1['Sentiment'].value_counts().rename_axis('Sentiment').reset_index(name='Count')
#df1.groupby(by=['Sentiment']).sum()
sentiment_plot = px.bar(
  sentiment_count,
  x = 'Count',
  y = 'Sentiment',
  orientation = "h",
  color_discrete_sequence=["#0083B8"]*len(sentiment_count)
)
sentiment_plot.update_layout(
  plot_bgcolor="rgba(0,0,0,0)"
)

from wordcloud   import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

stop_words_wc = stop_words
for word in ['no','not','nor']:
  stop_words_wc.add(word)

def wordcloud(string,color):
    wc = WordCloud(background_color=color, width=1200,height=650,mask=None,random_state=1,
                   max_font_size=200,stopwords=stop_words_wc,collocations=False,repeat = False).generate(string)
    fig=plt.figure(figsize=(20,8))
    plt.axis('off')
    plt.imshow(wc)

source_string = " ".join(df1['pr_text_swr_lemma'].astype('str'))
fig = wordcloud(source_string,'Black')


left_column, right_column = st.columns(2)
with left_column:
  st.subheader("Sentiment Counts of Extracted Tweets")
  st.plotly_chart(sentiment_plot)
with right_column:
  st.subheader("Word Cloud")
  st.pyplot(fig)

left_column, right_column = st.columns(2)
with left_column:
  if st.checkbox('Positive Tweets'):
    source_string1 = " ".join(df1[(df1.Sentiment=='Positive')]['pr_text_swr_lemma'].astype('str'))
    if source_string1 != "":
      fig1 = wordcloud(source_string1,'Black')
      st.subheader("Word cloud for Positive Tweets")
      st.pyplot(fig1, use_container_width=True)
with right_column:
  if st.checkbox('Negative Tweets'):
    source_string2 = " ".join(df1[(df1.Sentiment=='Negative')]['pr_text_swr_lemma'].astype('str'))
    if source_string2 != "":
      fig2 = wordcloud(source_string2,'Black')
      st.subheader("Word cloud for Negative Tweets")
      st.pyplot(fig2, use_container_width=True, clear_figure=False)

st.markdown("---")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline
tfidf_vectorizer1 = TfidfVectorizer(stop_words=stop_words, ngram_range=(2,3))

left_column, right_column = st.columns(2)
with left_column:
  st.subheader("Top 5 Topics using NMF Topic Modelling")
  nmf = NMF(n_components=5)
  pipe = make_pipeline(tfidf_vectorizer1, nmf)
  pipe.fit(df1['text'])
  def print_top_words(model, feature_names, n_top_words):
      for topic_idx, topic in enumerate(model.components_):
          message = "Topic #%d: " % (topic_idx+1)
          message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
          st.write(message)
      print()
  print_top_words(nmf, tfidf_vectorizer1.get_feature_names(), n_top_words=3)

with right_column:
  st.subheader("Top 5 Topics using LDA Topic Modelling")
  from sklearn.decomposition import LatentDirichletAllocation
  tfidf_vectorizer2 = TfidfVectorizer(stop_words=stop_words, ngram_range=(2,3))
  lda = LatentDirichletAllocation(n_components=5)
  pipe1 = make_pipeline(tfidf_vectorizer2, lda)
  pipe1.fit(df1['text'])
  def print_top_words(model, feature_names, n_top_words):
      for topic_idx, topic in enumerate(model.components_):
          message = "Topic #%d: " % (topic_idx+1)
          message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
          st.write(message)
      print()
  print_top_words(lda, tfidf_vectorizer2.get_feature_names(), n_top_words=3)

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_words(df1['pr_text_swr_lemma'], 15)
df2 = pd.DataFrame(common_words, columns = ['Unigram' , 'Count'])

def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words1 = get_top_n_bigram(df1['pr_text_swr_lemma'], 15)
df3 = pd.DataFrame(common_words1, columns = ['Bigram' , 'Count'])

def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words2 = get_top_n_trigram(df1['pr_text_swr_lemma'], 15)
df4 = pd.DataFrame(common_words2, columns = ['Trigram' , 'Count'])

left_column, middle_column, right_column = st.columns(3)
with left_column:
  st.subheader("Unigram")
  st.dataframe(df2, use_container_width=True)
with middle_column:
  st.subheader("Bigram")
  st.dataframe(df3, use_container_width=True)
with right_column:
  st.subheader("Trigram")
  st.dataframe(df4, use_container_width=True)



Overwriting app.py


In [12]:
!streamlit run app.py &>/dev/null&

In [13]:
!npx localtunnel --port 8501

npx: installed 22 in 2.896s
your url is: https://dull-teams-teach-34-132-126-143.loca.lt
^C


#Ignore





In [ ]:
!pip install snscrape

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def extract_tweets(n=5):
    tweets_list1 = []
    since_date = int((datetime.now()+timedelta(days=-1)).timestamp())
    until_date = int((datetime.now()).timestamp())
    i=-1
    try:
      n = abs(int(n))
    except Exception as e:
      print("Enter a valid number\n",e)
      raise
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(
            query='(covid OR covid19 OR coronavirus) min_replies:1 lang:en -filter:links -filter:replies until_time:{0} since_time:{1}'.format(until_date,since_date)
        ).get_items()
    ):
        tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.username])
        if((i + 1)==n):
            break
    else:
        print("Only {0} tweets available for past 24 hours".format(i+1))
    tweets_df1 = pd.DataFrame(
        tweets_list1, columns=["Datetime", "Tweet Id", "Text", "Username"]
    )
    return tweets_df1


In [ ]:
df= extract_tweets(n=5)

INFO:snscrape.modules.twitter:Retrieving guest token from search page
2022-10-06 15:19:09.841 Retrieving guest token from search page
INFO:snscrape.base:Retrieving https://twitter.com/search?f=live&lang=en&q=%28covid+OR+covid19+OR+coronavirus%29+min_replies%3A1+lang%3Aen+-filter%3Alinks+-filter%3Areplies+until_time%3A1665069549+since_time%3A1664983149&src=spelling_expansion_revert_click
2022-10-06 15:19:09.858 Retrieving https://twitter.com/search?f=live&lang=en&q=%28covid+OR+covid19+OR+coronavirus%29+min_replies%3A1+lang%3Aen+-filter%3Alinks+-filter%3Areplies+until_time%3A1665069549+since_time%3A1664983149&src=spelling_expansion_revert_click
INFO:snscrape.modules.twitter:Retrieving scroll page None
2022-10-06 15:19:10.061 Retrieving scroll page None
INFO:snscrape.base:Retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&inc

In [ ]:
df

,Datetime,Tweet Id,Text,Username
0,2022-10-06 15:18:06+00:00,1578041919019663361,Poll: I got my covid booster and flu shot this...,EmmaSManning
1,2022-10-06 15:17:19+00:00,1578041718741540865,"Wow, what an incredible team of people to help...",DrSdeG
2,2022-10-06 15:16:08+00:00,1578041423634599936,Myth: #Omicron is the mildest variant yet.\nTr...,Billius27
3,2022-10-06 15:15:29+00:00,1578041259112931330,How did people die before coronavirus was inve...,EzraKahan
4,2022-10-06 15:15:15+00:00,1578041199054802946,"Cw- death, covid\n\nFirst of I want to thank y...",And_Pixie_Dust


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import emoji as ej
import re
import string
# NLTK imports
import nltk
nltk.download('punkt')
# Download stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def emoji_to_text(text):
    if type(text) != float:
        return (ej.demojize(text)).replace(":", " ").replace("_"," ")
    else:
        return text.replace(":", " ").replace("_"," ")


def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    tweet = re.sub('#','',tweet)
    #emoj = re.compile("["
        #u"\U0001F600-\U0001F64F"  # emoticons
        #u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        #u"\U0001F680-\U0001F6FF"  # transport & map symbols
        #u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        #u"\U00002500-\U00002BEF"  # chinese char
        #u"\U00002702-\U000027B0"
        #u"\U00002702-\U000027B0"
        #u"\U000024C2-\U0001F251"
        #u"\U0001f926-\U0001f937"
        #u"\U00010000-\U0010ffff"
        #u"\u2640-\u2642" 
        #u"\u2600-\u2B55"
        #u"\u200d"
        #u"\u23cf"
        #u"\u23e9"
        #u"\u231a"
        #u"\ufe0f"  # dingbats
        #u"\u3030"
        #              "]+", re.UNICODE)
    #return re.sub(emoj, '', tweet)
    return tweet

def remove_mult_spaces(text): # remove multiple spaces
  return re.sub("\s\s+" , " ", text)

contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "i'd": "i would", "i'd've": "i would have","i'll": "i will",
                     "i'll've": "i will have","i'm": "i am","i've": "i have", "isn't": "is not",
                     "it'd": "it would","it’s":"it is","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are","what’s":"what is" ,"what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

def CleanFurther(Text):
  Text = re.sub(" \d+", " ", Text) # Numbers 
  Text = re.sub(r'(?:^| )\w(?:$| )', ' ', Text).strip() #Single letters 
  Text = re.sub('[%s]' % re.escape("""!"#$%&'’()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', Text)  # remove punctuations
  Text = re.sub('\s+', ' ',Text) #Extra whitespaces
  return Text

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

In [ ]:
def data_processing(df):
  # drop duplicates
  new_df = df.drop_duplicates(subset=['Tweet Id'], keep='first')
  new_df['text'] = new_df.Text.str.lower()
  new_df = new_df.drop(columns='Text')
  new_df = new_df[(new_df.text.str.contains('covid'))|(new_df.text.str.contains('coronavirus'))] #to ensure tweets have a reference to covid and are not extrcaed due to just the username having reference to covid
  new_df = new_df.reset_index(drop=True)
  new_df['text_ett'] = new_df['text'].apply(emoji_to_text)
  new_df['pr_text'] = new_df.text_ett.apply(lambda x:remove_usernames_links(x)).values
  new_df['pr_text']=new_df['pr_text'].apply(lambda x:expand_contractions(x))
  new_df['pr_text']=new_df['pr_text'].apply(lambda x:CleanFurther(x))
  new_df['pr_text_swr'] = new_df['pr_text'].apply(lambda x: ' '.join([str(w).strip('.') for w in word_tokenize(x) if not w in stop_words if not w in string.punctuation if not w in ['``',"''"]]))
  new_df['pr_text_swr_lemma'] = new_df.pr_text_swr.apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.split()]))
  return new_df

In [ ]:
df= data_processing(df)

In [ ]:
df

,Datetime,Tweet Id,Username,text,text_ett,pr_text,pr_text_swr,pr_text_swr_lemma
0,2022-10-06 15:18:06+00:00,1578041919019663361,EmmaSManning,poll: i got my covid booster and flu shot this...,poll i got my covid booster and flu shot this...,poll got my covid booster and flu shot this mo...,poll got covid booster flu shot morning thus c...,poll got covid booster flu shot morning thus c...
1,2022-10-06 15:17:19+00:00,1578041718741540865,DrSdeG,"wow, what an incredible team of people to help...","wow, what an incredible team of people to help...",wow what an incredible team of people to help ...,wow incredible team people help us evelina lon...,wow incredible team people help u evelina long...
2,2022-10-06 15:16:08+00:00,1578041423634599936,Billius27,myth: #omicron is the mildest variant yet.\ntr...,myth #omicron is the mildest variant yet.\ntr...,myth omicron is the mildest variant yet truth ...,myth omicron mildest variant yet truth omicron...,myth omicron mildest variant yet truth omicron...
3,2022-10-06 15:15:29+00:00,1578041259112931330,EzraKahan,how did people die before coronavirus was inve...,how did people die before coronavirus was inve...,how did people die before coronavirus was inve...,people die coronavirus invented,people die coronavirus invented
4,2022-10-06 15:15:15+00:00,1578041199054802946,And_Pixie_Dust,"cw- death, covid\n\nfirst of i want to thank y...","cw- death, covid\n\nfirst of i want to thank y...",cw death covid first of want to thank you all ...,cw death covid first want thank suggestions th...,cw death covid first want thank suggestion thi...


In [ ]:
st.dataframe(df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [ ]:
#!./ngrok authtokens 2FlYOMV8kjXbMWW6yrOvW4Vg0bX_5Dq97DVKWnyi5Ds3eU4cX

/bin/bash: ./ngrok: No such file or directory


In [ ]:
#!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.682s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
#!npx localtunnel --port 8501

npx: installed 22 in 1.739s
your url is: https://sixty-emus-win-35-188-231-115.loca.lt


2022-10-06 15:11:28.352 t=2022-10-06T15:11:28+0000 lvl=warn msg="failed to open private leg" id=2c8a390d9026 privaddr=localhost:80 err="dial tcp 127.0.0.1:80: connect: connection refused"
2022-10-06 15:11:28.668 t=2022-10-06T15:11:28+0000 lvl=warn msg="failed to open private leg" id=166c8c44a65a privaddr=localhost:80 err="dial tcp 127.0.0.1:80: connect: connection refused"
INFO:pyngrok.process.ngrok:t=2022-10-06T15:12:26+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
2022-10-06 15:12:26.553 t=2022-10-06T15:12:26+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


^C


In [ ]:
!streamlit run app.py

2022-10-06 15:22:38.540 INFO    numexpr.utils: NumExpr defaulting to 2 threads.



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.188.231.115:8501

  Stopping...
^C


In [ ]:
pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 25.5 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=5dc2eb86b0ea4e8da7acb7595ec9a4a2d0bc752f614e03941b926c6b10a0a49c
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ls

sample_data


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(port='8501')
public_url

INFO:pyngrok.ngrok:Opening tunnel named: http-80-b89ae37e-3c63-4e58-b424-5a50abcd09d9
2022-10-06 15:05:06.519 Opening tunnel named: http-80-b89ae37e-3c63-4e58-b424-5a50abcd09d9


INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:07+0000 lvl=info msg="no configuration paths supplied"
2022-10-06 15:05:07.331 t=2022-10-06T15:05:07+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:07+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-10-06 15:05:07.361 t=2022-10-06T15:05:07+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:07+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-10-06 15:05:07.374 t=2022-10-06T15:05:07+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:07+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2022-10-06 15:05:07.381 t=2022-10-06T15:05:07+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:

<NgrokTunnel: "http://2f81-35-188-231-115.ngrok.io" -> "http://localhost:80">

2022-10-06 15:05:07.535 t=2022-10-06T15:05:07+0000 lvl=info msg="started tunnel" obj=tunnels name="http-80-b89ae37e-3c63-4e58-b424-5a50abcd09d9 (http)" addr=http://localhost:80 url=http://2f81-35-188-231-115.ngrok.io
INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:07+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-b89ae37e-3c63-4e58-b424-5a50abcd09d9 addr=http://localhost:80 url=https://2f81-35-188-231-115.ngrok.io
2022-10-06 15:05:07.551 t=2022-10-06T15:05:07+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-b89ae37e-3c63-4e58-b424-5a50abcd09d9 addr=http://localhost:80 url=https://2f81-35-188-231-115.ngrok.io
INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:07+0000 lvl=info msg=end pg=/api/tunnels id=4f3b6b84c292ff49 status=201 dur=39.296986ms
2022-10-06 15:05:07.562 t=2022-10-06T15:05:07+0000 lvl=info msg=end pg=/api/tunnels id=4f3b6b84c292ff49 status=201 dur=39.296986ms
INFO:pyngrok.process.ngrok:t=2022-10-06T15:05:07+0000 lvl=info msg=start pg="/api/tunnels/http-80